# Download and process sentinel 2 data

## John Brandt
## April 1, 2020

## Package imports, API import, source scripts

In [1]:
import datetime
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import scipy.sparse as sparse
import seaborn as sns
import yaml

from collections import Counter
from osgeo import ogr, osr
from random import shuffle
from scipy.sparse.linalg import splu
from sentinelhub import WmsRequest, WcsRequest, MimeType
from sentinelhub import CRS, BBox, constants, DataSource, CustomUrlParam
from skimage.transform import resize

with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%matplotlib inline
%run ../src/preprocessing/slope.py
%run ../src/preprocessing/indices.py
%run ../src/downloading/utils.py
%run ../src/preprocessing/cloud_removal.py
%run ../src/preprocessing/whittaker_smoother.py
%run ../src/dsen2/utils/DSen2Net.py

Using TensorFlow backend.


## Parameters

In [2]:
# Parameters
SUPER_RESOLVE = True
YEAR = 2019
TIME = ('{}-12-01'.format(str(YEAR - 1)), '{}-02-01'.format(str(YEAR + 1)))
EPSG = CRS.WGS84
IMSIZE = 48
DATA_LOCATION = '../data/ghana-test.csv'
OUTPUT_FOLDER = '../data/test-smooth-200/'

# For DSen2 superresolve
MDL_PATH = "../src/dsen2/models/"
INPUT_SHAPE = ((4, None, None), (6, None, None))
MODEL = s2model(INPUT_SHAPE, num_layers=6, feature_size=128)
PREDICT_FILE = MDL_PATH+'s2_032_lr_1e-04.hdf5'
MODEL.load_weights(PREDICT_FILE)

# Constants
starting_days = np.cumsum([0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30])
c_arr = np.array([[1, 1, 1, 1, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 2, 3, 2, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 1, 1, 1, 1,],])
                  
c_arr = c_arr / 3
o_arr = 1 - c_arr
c_arr = np.tile(c_arr[:, :, np.newaxis], (1, 1, 11))
o_arr = np.tile(o_arr[:, :, np.newaxis], (1, 1, 11))

Instructions for updating:
Colocations handled automatically by placer.


# Helper functions

In [3]:
def DSen2(d10, d20):
    """Super resolves 20 meter bans using the DSen2 convolutional
       neural network, as specified in Lanaras et al. 2018
       https://github.com/lanha/DSen2

        Parameters:
         d10 (arr): (4, X, Y) shape array with 10 meter resolution
         d20 (arr): (6, X, Y) shape array with 20 meter resolution

        Returns:
         prediction (arr): (6, X, Y) shape array with 10 meter superresolved
                          output of DSen2 on d20 array
    """
    test = [d10, d20]
    input_shape = ((4, None, None), (6, None, None))
    prediction = _predict(test, input_shape, deep=False)
    return prediction

def _predict(test, input_shape, model = MODEL, deep=False, run_60=False):
    
    prediction = model.predict(test, verbose=1)
    return prediction

# Bounding boxes

In [4]:
def calc_bbox(plot_id, df):
    """ Calculates the corners of a bounding box from an input
        pandas dataframe as output by Collect Earth Online

        Parameters:
         plot_id (int): plot_id of associated plot
         df (pandas.DataFrame): dataframe of associated CEO survey
    
        Returns:
         bounding_box (list): [(min(x), min(y)),
                              (max(x), max_y))]
    """
    subs = df[df['PLOT_ID'] == plot_id]
    # TOP, LEFT, BOTTOM, RIGHT
    # (min x, min y), (max x, max y)
    return [(min(subs['LON']), min(subs['LAT'])),
            (max(subs['LON']), max(subs['LAT']))]

def bounding_box(points, expansion = 160):
    """ Calculates the corners of a bounding box with an
        input expansion in meters from a given bounding_box
        
        Subcalls:
         calculate_epsg, convertCoords

        Parameters:
         points (list): output of calc_bbox
         expansion (float): number of meters to expand or shrink the
                            points edges to be
    
        Returns:
         bl (tuple): x, y of bottom left corner with edges of expansion meters
         tr (tuple): x, y of top right corner with edges of expansion meters
    """
    bl = list(points[0])
    tr = list(points[1])
    inproj = Proj('epsg:4326')
    outproj_code = calculate_epsg(bl)
    outproj = Proj('epsg:' + str(outproj_code))
    
    bl_utm =  transform(inproj, outproj, bl[1], bl[0])
    tr_utm =  transform(inproj, outproj, tr[1], tr[0])

    distance1 = tr_utm[0] - bl_utm[0]
    distance2 = tr_utm[1] - bl_utm[1]
    expansion1 = (expansion - distance1)/2
    expansion2 = (expansion - distance2)/2
        
    bl_utm = [bl_utm[0] - expansion1, bl_utm[1] - expansion2]
    tr_utm = [tr_utm[0] + expansion1, tr_utm[1] + expansion2]

    #assert (tr_utm[0] - bl_utm[0]) == expansion
    #assert (tr_utm[1] - bl_utm[1]) == expansion

    zone = str(outproj_code)[3:]
    zone = zone[1:] if zone[0] == "0" else zone
    direction = 'N' if tr[1] >= 0 else 'S'
    utm_epsg = "UTM_" + zone + direction
    return (bl_utm, tr_utm), CRS[utm_epsg]

# Data download

In [5]:
def identify_clouds(bbox, epsg, time = TIME):

    for try_ in range(0, 5):
        try:
            box = BBox(bbox, crs = epsg)
            cloud_request = WsRequest(
                layer='CLOUD_NEW',
                bbox=box,
                time=time,
                width=96,
                height=96,
                image_format =  MimeType.TIFF_d8,
                maxcc=0.75,
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=48))
            
            shadow_request = WmsRequest(
                layer='SHADOW',
                bbox=box,
                time=time,
                width=96,
                height=96,
                image_format =  MimeType.TIFF_d16,
                maxcc=0.75,
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=48))

            cloud_img = cloud_request.get_data()
            cloud_img = np.array(cloud_img)

            if np.max(cloud_img > 10):
                cloud_img = cloud_img / 255
            assert np.max(cloud_img) <= 1.
            print("Cloud_probs shape: {}".format(cloud_img.shape))
            
            shadow_img = shadow_request.get_data()
            shadow_img = np.array(shadow_img)
            print("Shadows_shape: {}".format(shadow_img.shape))

            if np.max(shadow_img > 10):
                shadow_img = shadow_img / 65535
            print(np.max(shadow_img))
            
            shadows = mcm_shadow_mask(np.array(shadow_img), cloud_img)
            shadows = shadows[:, 24:-24, 24:-24]
            return cloud_img[:, 24:-24, 24:-24], shadows
        except Exception as e:
            logging.fatal(e, exc_info=True)
    
    
def download_dem(plot_id, df, epsg, image_format = MimeType.TIFF_d32f):
    """ Downloads MapZen digital elevation model and return slope

        Parameters:
         plot_id (tuple): plot id from collect earth online (CEO)
         df (pandas.DataFrame): data associated with plot_id from CEO
         epsg (int): UTM EPSG associated with plot_id
    
        Returns:
         slope (arr): (X, Y, 1) array of per-pixel slope from [0, 1]
    """
    location = calc_bbox(plot_id, df = df)
    bbox, epsg = bounding_box(location, expansion = (32+2)*10)
    box = BBox(bbox, crs = epsg)
    dem_request = WcsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=box,
                         resx = "10m",
                         resy = "10m",
                         instance_id=API_KEY,
                         image_format= image_format,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image_init = dem_request.get_data()[0]
    dem_image = np.copy(dem_image_init)
    dem_image = median_filter(dem_image_init, size = 5)
    slope = calcSlope(dem_image.reshape((1, 32+2, 32+2)),
                      np.full((32+2, 32+2), 10),
                      np.full((32+2, 32+2), 10), 
                      zScale = 1, minSlope = 0.02)
    slope = slope / 90
    slope = slope.reshape((32+2, 32+2, 1))
    #slope[slope > 0.25] = 0.25
    #slope[slope < 0.02] = 0.
    print(slope.shape)
    #slope[np.argwhere(slope < 0.05)] = 0.
    slope = slope[1:32+1, 1:32+1, :]
    return slope, dem_image_init

        
def download_layer(bbox, epsg, time = TIME, image_format = MimeType.TIFF_d16):
    """ Downloads the L2A sentinel layer with 10 and 20 meter bands
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         img (arr):
         img_request (obj): 
    """
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer='L2A20',
                bbox=box,
                time=time,
                image_format = image_format,
                data_source = DataSource.SENTINEL2_L2A,
                maxcc=0.75,
                resx='20m', resy='20m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=48),
            )
        img_bands = image_request.get_data()
        img_20 = np.stack(img_bands)

        if np.max(img_20) >= 10:
            img_20 = img_20 / 65535
        assert np.max(img_20) <= 2.

        s2_20_usage = (img_20.shape[1]*img_20.shape[2])/(512*512) * (6/3) * img_20.shape[0]
        print("Original 20 meter bands size: {}, using {} PU".format(img_20.shape, s2_20_usage))
        img_20 = resize(img_20, (img_20.shape[0], IMSIZE, IMSIZE, img_20.shape[-1]), order = 0)
        
        image_request = WcsRequest(
                layer='L2A10',
                bbox=box,
                time=time,
                image_format = image_format,
                data_source = DataSource.SENTINEL2_L2A,
                maxcc=0.75,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'BICUBIC',
                                    constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
                time_difference=datetime.timedelta(hours=48),
        )
        
        img_bands = image_request.get_data()
        img_10 = np.stack(img_bands)
        print("The original L2A image size is: {}".format(img_10.shape))
        img_10 = resize(img_10, (img_10.shape[0], IMSIZE, IMSIZE, img_10.shape[-1]), order = 0)
        img = np.concatenate([img_10, img_20], axis = -1)

        if np.max(img_10) >= 10:
            img_10 = img_10 / 65535
        assert np.max(img_10) <= 2.
        return img, image_request

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Cloud and shadow removal

In [6]:
def remove_missed_clouds(img):
    """ Removes steps that are likely to be missed cloud or shadows
        based on two interquartile ranges for the near infrared band
        
        Parameters:
         img (arr):

        Returns:
         to_remove (list): 
    """
    iqr = np.percentile(img[:, :, :, 3].flatten(), 75) - np.percentile(img[:, :, :, 3].flatten(), 25)
    thresh_t = np.percentile(img[:, :, :, 3].flatten(), 75) + iqr*2
    thresh_b = np.percentile(img[:, :, :, 3].flatten(), 25) - iqr*2
    diffs_fw = np.diff(img, 1, axis = 0)
    diffs_fw = np.mean(diffs_fw, axis = (1, 2, 3))
    diffs_fw = np.array([0] + list(diffs_fw))
    diffs_bw = np.diff(np.flip(img, 0), 1, axis = 0)
    diffs_bw = np.flip(np.mean(diffs_bw, axis = (1, 2, 3)))
    diffs_bw = np.array(list(diffs_bw) + [0])
    diffs = abs(diffs_fw - diffs_bw) * 100 # 3, -3 -> 6, -3, 3 -> 6, -3, -3
    #diffs = [int(x) for x in diffs]
    outlier_percs = []
    for step in range(img.shape[0]):
        bottom = len(np.argwhere(img[step, :, :, 3].flatten() > thresh_t))
        top = len(np.argwhere(img[step, :, :, 3].flatten() < thresh_b))
        p = 100* ((bottom + top) / (IMSIZE*IMSIZE))
        outlier_percs.append(p)
    to_remove = np.argwhere(np.array(outlier_percs) > 15)
    return to_remove

# Download function

In [11]:
def download_new_dem(data_location = DATA_LOCATION, output_folder = OUTPUT_FOLDER, image_format = MimeType.TIFF_d16):
    df = pd.read_csv(data_location)
    df.columns = [x.upper() for x in df.columns]
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG',
                  'IMAGERYMONTHPLANET', 'IMAGERYYEARPLANET', 'IMAGERYDATESECUREWATCH',
                  'IMAGERYENDDATESECUREWATCH', 'IMAGERYFEATUREPROFILESECUREWATCH',
                  'IMAGERYSTARTDATESECUREWATCH','IMAGERY_ATTRIBUTIONS',
                  'SAMPLE_GEOM']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    print(len(plot_ids))
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    to_download = [x for x in plot_ids if x not in existing]
    print("STARTING DOWNLOAD OF {} plots from {} to {}".format(len(to_download), data_location, output_folder))
    errors = []
    for i, val in enumerate(to_download):
        print("Downloading {}/{}, {}".format(i+1, len(to_download), val))
        initial_bbx = calc_bbox(val, df = df)
        dem_bbx, epsg = bounding_box(initial_bbx, expansion = 32*10)
        slope, dem = download_dem(val, epsg = epsg, df = df)
        print(dem.shape)
        np.save(output_folder + str(val), dem)
        np.save("../data/train-slope/" + str(val), slope)

In [ ]:
from scipy.ndimage import median_filter
for i in reversed(os.listdir("../data/train-csv/")):
    if "ethiopia" in i:
    #if ".csv" in i:
        #if any(x in i for x in ["africa-west", "cameroon", "koure", "niger"]):
        tile = download_new_dem("../data/train-csv/" + i, "../data/train-dem/", image_format = MimeType.TIFF_d16)

In [25]:
def download_plots(data_location = DATA_LOCATION, output_folder = OUTPUT_FOLDER, image_format = MimeType.TIFF_d16):
    """ Downloads slope and sentinel-2 data for all plots associated
        with an input CSV from a collect earth online survey
        
        Parameters:
         data_location (os.path)
         output_folder (os.path)
        
        Subcalls:
         calc_bbox, bounding_box
         identify_clouds, download_layer, check_zenith, download_dem
         remove_clouds_and_shadows, remove_missed_clouds
         DSen2
         calculate_and_save_best_images
         
        Creates:
         output_folder/{plot_id}.npy
    
        Returns:
         None
    """
    df = pd.read_csv(data_location)
    df.columns = [x.upper() for x in df.columns]
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG',
                  'IMAGERYMONTHPLANET', 'IMAGERYYEARPLANET', 'IMAGERYDATESECUREWATCH',
                  'IMAGERYENDDATESECUREWATCH', 'IMAGERYFEATUREPROFILESECUREWATCH',
                  'IMAGERYSTARTDATESECUREWATCH','IMAGERY_ATTRIBUTIONS',
                  'SAMPLE_GEOM']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    print(len(plot_ids))
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    to_download = [x for x in plot_ids if x not in existing]
    print("STARTING DOWNLOAD OF {} plots from {} to {}".format(len(to_download), data_location, output_folder))
    errors = []
    for i, val in enumerate(to_download):
        print("Downloading {}/{}, {}".format(i+1, len(to_download), val))
        initial_bbx = calc_bbox(val, df = df)
        sentinel2_bbx, epsg = bounding_box(initial_bbx, expansion = IMSIZE*10)
        cloud_bbx, _ = bounding_box(initial_bbx, expansion = 96*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            probs, shadows = identify_clouds(cloud_bbx, epsg = epsg)
            shadow_sums = np.sum(shadows, axis = (1, 2))
            shadow_steps = np.argwhere(shadow_sums > (48*48/3))
            dem, _ = download_dem(val, epsg = epsg, df = df)
            img, image_request = download_layer(sentinel2_bbx, epsg = epsg, image_format = image_format)

            #np.save("../data/raw/train-raw/" + str(val) + ".npy", img)
            #np.save("../data/raw/train-dates/" + str(val) + ".npy", image_request.get_dates())

            # Calculate imagery dates
            image_dates = []
            for date in image_request.get_dates():
                if date.year == YEAR - 1:
                    image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
                if date.year == YEAR:
                    image_dates.append(starting_days[(date.month-1)] + date.day)
                if date.year == YEAR + 1:
                    image_dates.append(365 + starting_days[(date.month-1)]+date.day)
            image_dates = np.array(image_dates)

            # Remove imagery where >4% is clouds, and where there is null data
            args = np.array([len(np.argwhere(probs[x].flatten() > 0.33)) for x in range(probs.shape[0])])
            dirty_steps = np.argwhere(args > (IMSIZE)*(IMSIZE) / 10)
            missing_images = [np.argwhere(img[x, :, : :].flatten() == 0.0) for x in range(img.shape[0])]
            missing_images = np.array([len(x) for x in missing_images])
            missing_images_p = [np.argwhere(img[x, :, : :].flatten() >= 1) for x in range(img.shape[0])]
            missing_images_p = np.array([len(x) for x in missing_images_p])
            missing_images += missing_images_p
            missing_images = list(np.argwhere(missing_images >= 25))
            to_remove = np.unique(np.array(list(dirty_steps) + list(missing_images) + list(shadow_steps)))

            # Remove null steps
            print("There are {}/{} dirty steps: {}"
                  " cloud, {} missing, {} shadow".format(len(to_remove),
                                                         len(img), len(dirty_steps),
                                                         len(missing_images),
                                                         #len(zenith_outliers),
                                                         len(shadow_steps)))

            img = np.delete(img, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)

            to_remove = remove_missed_clouds(img)
            img = np.delete(img, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            print("Removing {} steps based on ratio".format(len(to_remove)))



            x, interp = remove_cloud_and_shadows(img, probs, shadows, image_dates)
            to_remove = np.argwhere(np.mean(interp, axis = (1, 2, 3)) > 0.5)
            x = np.delete(x, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            print("Removing {} steps with >50% interpolation".format(len(to_remove)))
            
            x = x[:, 8:40, 8:40, :]
            print("Shape before super: {}".format(x.shape))

            d10 = x[:, :, :, 0:4]
            d20 = x[:, :, :, 4:10]

            d10 = np.swapaxes(d10, 1, -1)
            d10 = np.swapaxes(d10, 2, 3)
            d20 = np.swapaxes(d20, 1, -1)
            d20 = np.swapaxes(d20, 2, 3)
            superresolved = DSen2(d10, d20)
            superresolved = np.swapaxes(superresolved, 1, -1)
            superresolved = np.swapaxes(superresolved, 1, 2)

            # returns band IDXs 3, 4, 5, 7, 8, 9
            x[:, :, :, 4:10] = superresolved
            x = x[:, 8:24, 8:24, :]
            print("Shape after super: {}".format(x.shape))
            
            # Concatenate DEM
            dem = np.tile(dem.reshape((1, 32, 32, 1)), (x.shape[0], 1, 1, 1))
            dem = dem[:, 8:-8, 8:-8, :]
            x = np.concatenate([x, dem], axis = -1)
            x[:, :, :, -1] /= 90
            print(f"Shape after DEM: {x.shape}")

            # Calculate indices
            tiles = evi(x, True)
            tiles = bi(tiles, True)
            tiles = msavi2(tiles, True)
            x = si(tiles, True)
            
            missing_pixels = 0
            for band in range(0, 15):
                for time in range(0, x.shape[0]):
                    x_i = x[time, :, :, band]
                    missing_pixels += len(np.argwhere(np.isnan(x_i)))
                    x_i[np.argwhere(np.isnan(x_i))] = np.mean(x_i)
                    x[time, :, :, band] = x_i
            print("There are {} missing pixels".format(missing_pixels))

            # Interpolate linearly to 5 day frequency
            tiles, max_distance = calculate_and_save_best_images(x, image_dates)

            # Smooth linear interpolation
            print(f"There are {np.sum(np.isnan(tiles))} NA values")
            sm = Smoother(lmbd = 800, size = tiles.shape[0], nbands = 14, dim = 16)
            tiles = sm.interpolate_array(tiles)
            print(f"There are {np.sum(np.isnan(tiles))} NA values")
            
            if max_distance <= 240:
                np.save(output_folder + str(val), tiles)
                #np.save("../data/raw/train-clouds/" + str(val) + ".npy", probs)
                #np.save("../data/raw/train-shadows/" + str(val) + ".npy", shadows)
                print("Saved array of {} shape to {}".format(tiles.shape, val))
                print("\n")
            else:
                print("Skipping {} because there is a {} distance".format(val, max_distance))
                print("\n")

        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)
            #errors.append(img)
            #continue

In [26]:
for i in reversed(os.listdir("../data/train-csv/")):
    if "ethiopia" in i:
    #if ".csv" in i:
        #if any(x in i for x in ["africa-west", "cameroon", "koure", "niger"]):
        tile = download_plots("../data/train-csv/" + i, "../data/train-s2/", image_format = MimeType.TIFF_d16)

24
STARTING DOWNLOAD OF 23 plots from ../data/train-csv/ceo-ethiopia-finetune-sample-data.csv to ../data/train-s2/
Cloud_probs shape: (74, 96, 96)
Shadows_shape: (74, 96, 96, 3)
0.7515068284122988



(34, 34, 1)
Original 20 meter bands size: (74, 24, 24, 6), using 0.3251953125 PU
The original L2A image size is: (74, 48, 48, 4)
There are 18/74 dirty steps: 18 cloud, 0 missing, 0 shadow
Removing 0 steps based on ratio
Interpolated 35 px
Removing 2 steps with >50% interpolation
Shape before super: (54, 32, 32, 10)
54/54 [==============================] - 2s 29ms/step
Shape after super: (54, 16, 16, 10)
Shape after DEM: (54, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 35
There are 0 NA values
(72, 16, 16, 14)
(72, 72)
(72, 3584)
There are 0 NA values
Saved array of (24, 16, 16, 15) shape to 138444860


Cloud_probs shape: (74, 96, 96)
Shadows_shape: (74, 96, 96, 3)
1.0



(34, 34, 1)
Original 20 meter bands size: (74, 24, 24, 6), using 0.3251953125 PU
The original L2A image size is: (74, 48, 48, 4)
There are 23/74 dirty steps: 22 cloud, 2 missing, 1 shadow
Removing 8 steps based on ratio
Interpolated 0 px
Removing 0 steps with >50% interpolation
Shape before super: (43, 32, 32, 10)
43/43 [==============================] - 1s 29ms/step
Shape after super: (43, 16, 16, 10)
Shape after DEM: (43, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 125
0 {'image_date': [2], 'image_ratio': [1], 'image_idx': [5]}
5 {'image_date': [7], 'image_ratio': [1], 'image_idx': [6]}
10 {'image_date': [12], 'image_ratio': [1], 'image_idx': [7]}
15 {'image_date': [17], 'image_ratio': [1], 'image_idx': [8]}
20 {'image_date': [22], 'image_ratio': [1], 'image_idx': [9]}
25 {'image_date': [27], 'image_ratio': [1], 'image_idx': [10]}
30 {'image_date': [32], 'image_ratio': [1], 'image_idx': [11]}
35 {'image_date': [32], 'image_ratio': [1], 'image_idx': [11]}
40 {'image


(34, 34, 1)
Original 20 meter bands size: (74, 24, 24, 6), using 0.3251953125 PU
The original L2A image size is: (74, 48, 48, 4)
There are 25/74 dirty steps: 24 cloud, 6 missing, 1 shadow
Removing 1 steps based on ratio
Interpolated 128 px
Removing 4 steps with >50% interpolation
Shape before super: (44, 32, 32, 10)
44/44 [==============================] - 1s 29ms/step
Shape after super: (44, 16, 16, 10)
Shape after DEM: (44, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 85
0 {'image_date': [2], 'image_ratio': [1], 'image_idx': [4]}
5 {'image_date': [7], 'image_ratio': [1], 'image_idx': [5]}
10 {'image_date': [12], 'image_ratio': [1], 'image_idx': [6]}
15 {'image_date': [17], 'image_ratio': [1], 'image_idx': [7]}
20 {'image_date': [22], 'image_ratio': [1], 'image_idx': [8]}
25 {'image_date': [27], 'image_ratio': [1], 'image_idx': [9]}
30 {'image_date': [32], 'image_ratio': [1], 'image_idx': [10]}
35 {'image_date': [32], 'image_ratio': [1], 'image_idx': [10]}
40 {'image


(34, 34, 1)
Original 20 meter bands size: (74, 24, 24, 6), using 0.3251953125 PU
The original L2A image size is: (74, 48, 48, 4)
There are 19/74 dirty steps: 19 cloud, 3 missing, 0 shadow
Removing 1 steps based on ratio
Interpolated 0 px
Removing 0 steps with >50% interpolation
Shape before super: (54, 32, 32, 10)
54/54 [==============================] - 2s 28ms/step
Shape after super: (54, 16, 16, 10)
Shape after DEM: (54, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 60
0 {'image_date': [2], 'image_ratio': [1], 'image_idx': [5]}
5 {'image_date': [7], 'image_ratio': [1], 'image_idx': [6]}
10 {'image_date': [12], 'image_ratio': [1], 'image_idx': [7]}
15 {'image_date': [17], 'image_ratio': [1], 'image_idx': [8]}
20 {'image_date': [22], 'image_ratio': [1], 'image_idx': [9]}
25 {'image_date': [27], 'image_ratio': [1], 'image_idx': [10]}
30 {'image_date': [32], 'image_ratio': [1], 'image_idx': [11]}
35 {'image_date': [32], 'image_ratio': [1], 'image_idx': [11]}
40 {'image_

...


(34, 34, 1)
Original 20 meter bands size: (66, 24, 24, 6), using 0.2900390625 PU
The original L2A image size is: (66, 48, 48, 4)
There are 39/66 dirty steps: 38 cloud, 14 missing, 4 shadow
Removing 0 steps based on ratio
Interpolated 72 px
Removing 2 steps with >50% interpolation
Shape before super: (25, 32, 32, 10)
25/25 [==============================] - 1s 28ms/step
Shape after super: (25, 16, 16, 10)
Shape after DEM: (25, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 135
0 {'image_date': [4], 'image_ratio': [1], 'image_idx': [3]}
5 {'image_date': [4], 'image_ratio': [1], 'image_idx': [3]}
10 {'image_date': [9], 'image_ratio': [1], 'image_idx': [4]}
15 {'image_date': [14], 'image_ratio': [1], 'image_idx': [5]}
20 {'image_date': [19], 'image_ratio': [1], 'image_idx': [6]}
25 {'image_date': [24], 'image_ratio': [1], 'image_idx': [7]}
30 {'image_date': [29], 'image_ratio': [1], 'image_idx': [8]}
35 {'image_date': [34], 'image_ratio': [1], 'image_idx': [9]}
40 {'image_d


(34, 34, 1)
Original 20 meter bands size: (72, 24, 24, 6), using 0.31640625 PU
The original L2A image size is: (72, 48, 48, 4)
There are 39/72 dirty steps: 37 cloud, 10 missing, 5 shadow
Removing 0 steps based on ratio
Interpolated 198 px
Removing 8 steps with >50% interpolation
Shape before super: (25, 32, 32, 10)
25/25 [==============================] - 1s 31ms/step
Shape after super: (25, 16, 16, 10)
Shape after DEM: (25, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 180
0 {'image_date': [4], 'image_ratio': [1], 'image_idx': [4]}
5 {'image_date': [4], 'image_ratio': [1], 'image_idx': [4]}
10 {'image_date': [9], 'image_ratio': [1], 'image_idx': [5]}
15 {'image_date': [14], 'image_ratio': [1], 'image_idx': [6]}
20 {'image_date': [19], 'image_ratio': [1], 'image_idx': [7]}
25 {'image_date': [24], 'image_ratio': [1], 'image_idx': [8]}
30 {'image_date': [29], 'image_ratio': [1], 'image_idx': [9]}
35 {'image_date': [34], 'image_ratio': [1], 'image_idx': [10]}
40 {'image_d


(34, 34, 1)
Original 20 meter bands size: (72, 24, 24, 6), using 0.31640625 PU
The original L2A image size is: (72, 48, 48, 4)
There are 45/72 dirty steps: 39 cloud, 11 missing, 8 shadow
Removing 1 steps based on ratio
Interpolated 35 px
Removing 1 steps with >50% interpolation
Shape before super: (25, 32, 32, 10)
25/25 [==============================] - 1s 28ms/step
Shape after super: (25, 16, 16, 10)
Shape after DEM: (25, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 90
0 {'image_date': [4], 'image_ratio': [1], 'image_idx': [3]}
5 {'image_date': [4], 'image_ratio': [1], 'image_idx': [3]}
10 {'image_date': [9], 'image_ratio': [1], 'image_idx': [4]}
15 {'image_date': [14], 'image_ratio': [1], 'image_idx': [5]}
20 {'image_date': [19], 'image_ratio': [1], 'image_idx': [6]}
25 {'image_date': [24], 'image_ratio': [1], 'image_idx': [7]}
30 {'image_date': [29], 'image_ratio': [1], 'image_idx': [8]}
35 {'image_date': [34], 'image_ratio': [1], 'image_idx': [9]}
40 {'image_date


(34, 34, 1)
Original 20 meter bands size: (72, 24, 24, 6), using 0.31640625 PU
The original L2A image size is: (72, 48, 48, 4)
There are 33/72 dirty steps: 30 cloud, 5 missing, 5 shadow
Removing 1 steps based on ratio
Interpolated 0 px
Removing 0 steps with >50% interpolation
Shape before super: (38, 32, 32, 10)
38/38 [==============================] - 1s 29ms/step
Shape after super: (38, 16, 16, 10)
Shape after DEM: (38, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 40
There are 0 NA values
(72, 16, 16, 14)
(72, 72)
(72, 3584)
There are 0 NA values
Saved array of (24, 16, 16, 15) shape to 138444900


Cloud_probs shape: (66, 96, 96)
Shadows_shape: (66, 96, 96, 3)
0.9886015106431678



(34, 34, 1)
Original 20 meter bands size: (66, 24, 24, 6), using 0.2900390625 PU
The original L2A image size is: (66, 48, 48, 4)
There are 32/66 dirty steps: 31 cloud, 5 missing, 7 shadow
Removing 1 steps based on ratio
Interpolated 56 px
Removing 2 steps with >50% interpolation
Shape before super: (31, 32, 32, 10)
31/31 [==============================] - 1s 30ms/step
Shape after super: (31, 16, 16, 10)
Shape after DEM: (31, 16, 16, 11)
There are 0 missing pixels
Maximum time distance: 65
0 {'image_date': [4], 'image_ratio': [1], 'image_idx': [3]}
5 {'image_date': [4], 'image_ratio': [1], 'image_idx': [3]}
10 {'image_date': [9], 'image_ratio': [1], 'image_idx': [4]}
15 {'image_date': [14], 'image_ratio': [1], 'image_idx': [5]}
20 {'image_date': [19], 'image_ratio': [1], 'image_idx': [6]}
25 {'image_date': [24], 'image_ratio': [1], 'image_idx': [7]}
30 {'image_date': [29], 'image_ratio': [1], 'image_idx': [8]}
35 {'image_date': [34], 'image_ratio': [1], 'image_idx': [9]}
40 {'image_dat